In [1]:
import sys
print("Python Version:")
print(sys.version)
print("\nPython Executable Path:")
print(sys.executable)

Python Version:
3.9.23 (main, Jun  5 2025, 13:25:08) [MSC v.1929 64 bit (AMD64)]

Python Executable Path:
c:\Users\User\anaconda3\envs\tf-gpu\python.exe


In [2]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# 如果上面 > 0, 尝试在 GPU 上执行一个简单操作
if tf.test.is_gpu_available():
    print("GPU is available.")
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)
        print("Matrix multiplication on GPU:")
        print(c)
else:
    print("GPU is not available.")

TensorFlow Version: 2.10.0
Num GPUs Available: 1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available.
Matrix multiplication on GPU:
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
#load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [5]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255

x_train.shape

(60000, 28, 28, 1)

In [6]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [7]:
datagen = ImageDataGenerator(
    rotation_range=10, 
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(x_train)

In [8]:
# 4. 构建 CNN 模型
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),  # 防止过拟合
    Dense(10, activation="softmax")
])

In [9]:
# 5. 编译模型
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [10]:
# 6. 训练模型
import scipy

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=10,
    validation_data=(x_test, y_test)
)

Epoch 1/10
938/938 [==============================] - 23s 12ms/step - loss: 0.4464 - accuracy: 0.8587 - val_loss: 0.0547 - val_accuracy: 0.9819
Epoch 2/10
938/938 [==============================] - 10s 11ms/step - loss: 0.1792 - accuracy: 0.9465 - val_loss: 0.0381 - val_accuracy: 0.9861
Epoch 3/10
938/938 [==============================] - 11s 12ms/step - loss: 0.1341 - accuracy: 0.9599 - val_loss: 0.0267 - val_accuracy: 0.9906
Epoch 4/10
938/938 [==============================] - 13s 13ms/step - loss: 0.1103 - accuracy: 0.9675 - val_loss: 0.0240 - val_accuracy: 0.9914
Epoch 5/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0966 - accuracy: 0.9709 - val_loss: 0.0228 - val_accuracy: 0.9915
Epoch 6/10
938/938 [==============================] - 11s 12ms/step - loss: 0.0904 - accuracy: 0.9732 - val_loss: 0.0230 - val_accuracy: 0.9927
Epoch 7/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0793 - accuracy: 0.9777 - val_loss: 0.0194 - val_accuracy:

In [11]:
# 7. 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("测试集准确率:", test_acc)

测试集准确率: 0.9934999942779541
